<a href="https://colab.research.google.com/github/irfan00785/WHISPERED-TO-VOICED-SPEECH-CONVERSION-WITH-GANS/blob/main/Wishpered_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ***Liabraries To Import***


In [ ]:
#!/usr/bin/python3
from __future__ import print_function
from urllib.request import urlopen
from zipfile import ZipFile
from collections import namedtuple, OrderedDict
from subprocess import call
import glob
import os
import librosa   
import soundfile as sf
import tensorflow as tf
import numpy as np
import scipy.io.wavfile as wavfile
import argparse
import codecs
import timeit
import struct
import toml
import re
import sys
import time



SAMPLE_RATE = 16000 

### ***Clean Vocie File Downloading and Processing***


In [ ]:
zipurl = 'https://datashare.ed.ac.uk/bitstream/handle/10283/1942/clean_trainset_wav.zip?sequence=1&isAllowed=y'
    # Download the file from the URL
zipresp = urlopen(zipurl)
    # Create a new file on the hard drive
tempzip = open("/tmp/tempfile.zip", "wb")
    # Write the contents of the downloaded file into the new file
tempzip.write(zipresp.read())
    # Close the newly-created file
tempzip.close()
    # Re-open the newly-created file with ZipFile()
zf = ZipFile("/tmp/tempfile.zip")
    # Extract its contents into <extraction_path>
    # note that extractall will automatically create the path
zf.extractall(path = 'drive/MyDrive/Wishpered dataset/clean_trainset_wav/')
    # close the ZipFile instance
zf.close()

In [ ]:

#os.mkdir('drive/MyDrive/Wishpered dataset/sampled_testset_wav')

file_location = os.path.join('drive','MyDrive','Wishpered dataset','clean_trainset_wav', '*.wav')
print(file_location)

filenames = glob.glob(file_location)
print(filenames)
count = 0
for f in filenames:
  y, sr = librosa.load(f)
  data = librosa.resample(y, sr, SAMPLE_RATE)
  count += 1
  sf.write('drive/MyDrive/Wishpered dataset/sampled_testset_wav/'+os.path.basename(f), data, SAMPLE_RATE)
  

### ***Noisy Vocie File Downloading and Processing***

In [ ]:
zipurl = 'https://datashare.ed.ac.uk/bitstream/handle/10283/1942/noisy_trainset_wav.zip?sequence=2&isAllowed=y'
    # Download the file from the URL
zipresp = urlopen(zipurl)
    # Create a new file on the hard drive
tempzip = open("/tmp/tempfile.zip", "wb")
    # Write the contents of the downloaded file into the new file
tempzip.write(zipresp.read())
    # Close the newly-created file
tempzip.close()
    # Re-open the newly-created file with ZipFile()
zf = ZipFile("/tmp/tempfile.zip")
    # Extract its contents into <extraction_path>
    # note that extractall will automatically create the path
zf.extractall(path = 'drive/MyDrive/Wishpered dataset/noisy_trainset_wav/')
    # close the ZipFile instance
zf.close()

In [ ]:
#os.mkdir('drive/MyDrive/Wishpered dataset/sampled_noisy_trainset_wav')

file_location = os.path.join('drive','MyDrive','Wishpered dataset','clean_trainset_wav', '*.wav')
print(file_location)

filenames = glob.glob(file_location)
print(filenames)

count = 0
for f in filenames:
  y, sr = librosa.load(f)
  data = librosa.resample(y, sr, SAMPLE_RATE)
  count += 1
  sf.write('drive/MyDrive/Wishpered dataset/sampled_noisy_trainset_wav/'+os.path.basename(f), data, SAMPLE_RATE)


### ***Input Pipeline Creation***

In [ ]:
def _int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [ ]:
def slice_signal(signal, window_size, stride=0.5):
    """ Return windows of the given signal by sweeping in stride fractions
        of window
    """
    assert signal.ndim == 1, signal.ndim
    n_samples = signal.shape[0]
    offset = int(window_size * stride)
    slices = []
    for beg_i, end_i in zip(range(0, n_samples, offset),
                            range(window_size, n_samples + offset,
                                  offset)):
        if end_i - beg_i < window_size:
            break
        slice_ = signal[beg_i:end_i]
        if slice_.shape[0] == window_size:
            slices.append(slice_)
    return np.array(slices, dtype=np.int32)

In [ ]:
def read_and_slice(filename, wav_canvas_size, stride=0.5):
    fm, wav_data = wavfile.read(filename)
    if fm != 16000:
        raise ValueError('Sampling rate is expected to be 16kHz!')
    signals = slice_signal(wav_data, wav_canvas_size, stride)
    return signals

In [ ]:
def encoder_proc(wav_filename, noisy_path, out_file, wav_canvas_size):
    """ Read and slice the wav and noisy files and write to TFRecords.
        out_file: TFRecordWriter.
    """
    ppath, wav_name = os.path.split(wav_filename)
    noisy_filename = os.path.join(noisy_path, wav_name)
    wav_signals = read_and_slice(wav_filename, wav_canvas_size)
    noisy_signals = read_and_slice(noisy_filename, wav_canvas_size)
    assert wav_signals.shape == noisy_signals.shape, noisy_signals.shape

    for (wav, noisy) in zip(wav_signals, noisy_signals):
        wav_raw = wav.tobytes()
        noisy_raw = noisy.tobytes()
        example = tf.train.Example(features=tf.train.Features(feature={'wav_raw': _bytes_feature(wav_raw),'noisy_raw': _bytes_feature(noisy_raw)}))
        out_file.write(example.SerializeToString())
        

In [ ]:
save_path ='drive/MyDrive/Wishpered dataset/data/'
out_file = 'segan.tfrecords'
force_gen = True
cfg ='./e2e_maker.cfg'

if not os.path.exists(save_path):
  # make save path if it does not exist
  os.makedirs(save_path)
# set up the output filepath
out_filepath = os.path.join(save_path, out_file)
if os.path.splitext(out_filepath)[1] != '.tfrecords':
  # if wrong extension or no extension appended, put .tfrecords
  out_filepath += '.tfrecords'
else:
  out_filename, ext = os.path.splitext(out_filepath)
  out_filepath = out_filename + ext

# check if out_file exists and if force flag is set
if os.path.exists(out_filepath) and not force_gen:
  raise ValueError('ERROR: {} already exists. Set force flag (--force-gen) to '
         'overwrite. Skipping this speaker.'.format(out_filepath))
elif os.path.exists(out_filepath) and force_gen:
  print('Will overwrite previously existing tfrecords')
  os.unlink(out_filepath)

count = 0
with open(cfg) as cfh:
  # read the configuration description
  cfg_desc = toml.loads(cfh.read())
  beg_enc_t = timeit.default_timer()
  out_file = tf.io.TFRecordWriter(out_filepath)
  for dset_i, (dset, dset_desc) in enumerate(cfg_desc.items()):
    print('-' * 50)
    wav_dir = dset_desc['clean']
    wav_files = [os.path.join(wav_dir, wav) for wav in
                           os.listdir(wav_dir) if wav.endswith('.wav')]
    noisy_dir = dset_desc['noisy']
    nfiles = len(wav_files)
    for m, wav_file in enumerate(wav_files):
      sys.stdout.flush()
      print('Processing wav file {}/{} {}{}'.format(m + 1,nfiles,wav_file,' ' * 10))
      encoder_proc(wav_file, noisy_dir, out_file, 2 ** 14)
      count += 1

### ***Model Building***

In [ ]:
from tensorflow.python.client import device_lib

save_path = "segan_results"
synthesis_path = "dwavegan_samples"
model = 'gan'

if not os.path.exists(save_path):
        os.makedirs(save_path)           #Path to save out model files. (Def: dwavegan_model). 
if not os.path.exists(synthesis_path):
        os.makedirs(synthesis_path)      #Path to save output generated samples.(Def: dwavegan_samples)


devices = device_lib.list_local_devices()
udevices = []
for device in devices:
        if len(devices) > 1 and 'cpu' in device.name:
            # Use cpu only when we dont have gpus
            continue
        print('Using device: ', device.name)
        udevices.append(device.name)

if model == 'gan':
  print('Creating GAN model')
  se_model = SEGAN(sess, FLAGS, udevices)

### ***Data Loader***

In [133]:
def pre_emph(x, coeff=0.95):
    x0 = tf.reshape(x[0], [1,])
    diff = x[1:] - coeff * x[:-1]
    concat = tf.concat([x0, diff], 0)
    return concat


def read_and_decode(filename_queue, canvas_size, preemph=0.):
  wave = 0
  noisy = 0
  reader = tf.compat.v1.TFRecordReader()
  _, serialized_example = reader.read(filename_queue)
  features = tf.io.parse_single_example(
            serialized_example,
            features={
                'wav_raw': tf.io.FixedLenFeature([], tf.string),
                'noisy_raw': tf.io.FixedLenFeature([], tf.string),
            })
  wave = tf.io.decode_raw(features['wav_raw'], tf.int32)
  wave.set_shape(canvas_size)
  wave = (2./65535.) * tf.cast((wave - 32767), tf.float32) + 1.
  noisy = tf.io.decode_raw(features['noisy_raw'], tf.int32)
  noisy.set_shape(canvas_size)
  noisy = (2./65535.) * tf.cast((noisy - 32767), tf.float32) + 1.

  
  if preemph > 0:
        wave = tf.cast(pre_emph(wave, preemph), tf.float32)
        noisy = tf.cast(pre_emph(noisy, preemph), tf.float32)
  return wave, noisy

In [149]:
e2e_dataset = "drive/MyDrive/Wishpered dataset/data/segan.tfrecords"
tf.compat.v1.disable_eager_execution()

filename_queue = tf.compat.v1.train.string_input_producer([e2e_dataset])
canvas_size = 2**14
preemph = 0.95
batch_size = 150
g_enc_depths = [16, 32, 32, 64, 64, 128, 128, 256, 256, 512, 1024]
g_dilated_blocks = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512]

Gs = []
zs = []
gtruth_wavs = []
gtruth_noisy = []


get_wav, get_noisy = read_and_decode(filename_queue, canvas_size, preemph)

wavbatch, noisybatch = tf.compat.v1.train.shuffle_batch([get_wav,
                                             get_noisy],
                                             batch_size=batch_size,
                                             num_threads=2,
                                             capacity=1000 + 3 * batch_size,
                                             min_after_dequeue=1000,
                                             name='wav_and_noisy')

gtruth_wavs.append(wavbatch)
gtruth_noisy.append(noisybatch)

# add channels dimension to manipulate in D and G
wavbatch = tf.expand_dims(wavbatch, -1)
noisybatch = tf.expand_dims(noisybatch, -1)



## ***MODEL GENERATION***

In [157]:
#Dilated Convolution 

def atrous_conv1d(value, dilation, kwidth=3, num_kernels=1,
                  name='atrous_conv1d', bias_init=None, stddev=0.02):
    input_shape = value.get_shape().as_list()
    in_channels = input_shape[-1]
    assert len(input_shape) >= 3
    with tf.compat.v1.variable_scope(name):
        weights_init = tf.compat.v1.truncated_normal_initializer(stddev=0.02)
        # filter shape: [kwidth, in_channels, output_channels]
        filter_ = tf.compat.v1.get_variable('w', [kwidth, in_channels, num_kernels],initializer=weights_init,)
        padding = [[0, 0], [(kwidth/2) * dilation, (kwidth/2) * dilation],[0, 0]]

        #print(value)
        padded = tf.compat.v1.pad(value, padding, mode='SYMMETRIC')
        if dilation > 1:
            transformed = time_to_batch(padded, dilation)
            conv = tf.nn.conv1d(transformed, filter_, stride=1, padding='SAME')
            restored = batch_to_time(conv, dilation)
        else:
            restored = tf.nn.conv1d(padded, filter_, stride=1, padding='SAME')
        # Remove excess elements at the end.
        result = tf.slice(restored,
                          [0, 0, 0],
                          [-1, input_shape[1], num_kernels])
        if bias_init is not None:
            b = tf.get_variable('b', [num_kernels],
                                initializer=tf.constant_initializer(bias_init))
            result = tf.add(result, b)
        return result

In [158]:
#Residual Block

def residual_block(input_, dilation, kwidth, num_kernels=1,bias_init=None, stddev=0.02, do_skip=True,name='residual_block'):
  print('input shape to residual block: ', input_.get_shape())
  with tf.compat.v1.variable_scope(name):
    #h_a = tf.keras.layers.Conv1D(dilation_rate = dilation, kernel_size = kwidth, filters=num_kernels,
                            #bias_initializer=bias_init,input_shape=input_.get_shape().as_list())(input_)

    h_a = atrous_conv1d(input_, dilation, kwidth, num_kernels,
                            bias_init=bias_init, stddev=stddev)
    h = tf.tanh(h_a)

In [146]:
#Shaping the z value
def make_z(shape,is_ref, mean=0., std=1., name='z'):
  if is_ref:
    with tf.variable_scope(name) as scope:
      z_init = tf.random_normal_initializer(mean=mean, stddev=std)
      z = tf.get_variable("z", shape,initializer=z_init,trainable=False)
      if z.device != "/device:GPU:0":
        # this has to be created into gpu0
        print('z.device is {}'.format(z.device))
        assert False
  else:
    z = tf.random.normal(shape, mean=mean, stddev=std,name=name, dtype=tf.float32)
  return z

In [152]:
#Generator Model Defination

def generator(noisy_w, is_ref, spk=None, do_prelu = False):
  tf.compat.v1.get_variable_scope().reuse_variables()

  print('*** Building Generator ***')
  in_dims = noisy_w.get_shape().as_list()
  h_i = noisy_w
  if len(in_dims) == 2:
    h_i = tf.expand_dims(noisy_w, -1)
  elif len(in_dims) < 2 or len(in_dims) > 3:
    raise ValueError('Generator input must be 2-D or 3-D')

  kwidth = 3
  z = make_z([batch_size, h_i.get_shape().as_list()[1],g_enc_depths[-1]],is_ref)
  h_i = tf.concat([h_i, z],2)
  skip_out = True
  skips = []
  for block_idx, dilation in enumerate(g_dilated_blocks):
    name = 'g_residual_block_{}'.format(block_idx)
    if block_idx >= len(g_dilated_blocks) - 1:
      skip_out = False
    if skip_out:
      skip_i = residual_block(h_i,dilation, kwidth, num_kernels=32, bias_init=None, stddev=0.02,do_skip = True,name=name)
    


In [ ]:
g_nl = 'prelu'

prelu = False
if g_nl == 'prelu':
  prelu = True

z  = generator(noisybatch, is_ref=False, spk=None, do_prelu=prelu)
